<a href="https://colab.research.google.com/github/BMKEITA/AVD-RGB/blob/main/AVD_RGB_Detector_modulartesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
import matplotlib.pyplot as plt
from skimage import color
import os
from glob import glob
from PIL import Image
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

class AVDRGB_Detector:
    def __init__(self, use_spectral=True, use_uncertainty=True):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.use_spectral = use_spectral
        self.use_uncertainty = use_uncertainty
        self._initialize_parameters()
        if self.use_uncertainty:
            self.uncertainty_model = nn.Sequential(
                nn.Conv2d(3, 8, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2),
                nn.Conv2d(8, 16, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2),
                nn.Flatten(),
                nn.Linear(16*56*56, 10)
            ).to(self.device)
        print(f"AVD-RGB initialized on {self.device} with spectral={self.use_spectral}, uncertainty={self.use_uncertainty}")

    def _initialize_parameters(self):
        self.base_threshold = 0.5
        self.uncertainty_thresh = 0.7
        self.min_patch_size = 64

    def _rgb_spectral_analysis(self, image):
        lab = color.rgb2lab(image)
        l_channel = lab[:, :, 0]
        grad_maps = []
        for ksize in [3, 5]:
            grad_x = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=ksize)
            grad_y = cv2.Sobel(l_channel, cv2.CV_64F, 0, 1, ksize=ksize)
            grad_maps.append(np.sqrt(grad_x**2 + grad_y**2))
        combined_grad = np.mean(grad_maps, axis=0)
        normalized_grad = (combined_grad - combined_grad.min()) / (combined_grad.max() - combined_grad.min() + 1e-8)
        image_entropy = np.std(normalized_grad)
        dynamic_thresh = self.base_threshold + 0.2 * (1 - np.exp(-image_entropy/0.3))
        mask = (normalized_grad > dynamic_thresh).astype(np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        confidence = np.mean(normalized_grad[mask == 1]) if np.any(mask) else 0
        return mask, confidence

    def _uncertainty_estimation(self, image):
        if not self.use_uncertainty:
            return 0
        self.uncertainty_model.train()
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3),
        ])
        image = Image.fromarray(image)
        img_tensor = transform(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            outputs = torch.stack([self.uncertainty_model(img_tensor) for _ in range(8)])
        return outputs.var(dim=0).mean().item() * 2

    def _analyze_patches(self, mask):
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
        if num_labels < 2:
            return 0, None, None
        largest_component = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1
        largest_area = stats[largest_component, cv2.CC_STAT_AREA]
        largest_bbox = stats[largest_component, :4]
        largest_mask = (labels == largest_component).astype(np.uint8)
        valid_patches = 0
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= self.min_patch_size:
                valid_patches += 1
        return valid_patches, largest_mask, largest_bbox

    def detect(self, image):
        if self.use_spectral:
            spectral_mask, spectral_conf = self._rgb_spectral_analysis(image)
        else:
            spectral_mask = np.zeros(image.shape[:2], dtype=np.uint8)
            spectral_conf = 0

        uncertainty = self._uncertainty_estimation(image) if self.use_uncertainty else 0
        patch_count, largest_mask, largest_bbox = self._analyze_patches(spectral_mask)
        largest_area = largest_bbox[2] * largest_bbox[3] if largest_bbox is not None else 0

        if self.use_spectral and self.use_uncertainty:
            combined_conf = 0.7 * spectral_conf + 0.3 * uncertainty
        elif self.use_spectral:
            combined_conf = spectral_conf
        else:
            combined_conf = uncertainty

        is_adversarial = (combined_conf > 0.3) or (uncertainty > self.uncertainty_thresh and patch_count > 0)

        vis = image.copy()
        vis[spectral_mask == 1] = [255, 0, 0]
        if largest_mask is not None:
            vis[largest_mask == 1] = [0, 255, 0]

        return {
            'is_adversarial': bool(is_adversarial),
            'confidence': float(np.clip(combined_conf, 0, 1)),
            'patch_mask': spectral_mask,
            'visualization': vis,
            'metrics': {
                'spectral_confidence': float(spectral_conf),
                'uncertainty_score': float(uncertainty),
                'patch_count': int(patch_count),
                'largest_patch_area': int(largest_area),
                'largest_patch_bbox': tuple(largest_bbox) if largest_bbox is not None else None
            }
        }

def analyze_and_save_results(image_path, detector, output_dir="results", csv_writer=None, config_name="unknown"):
    os.makedirs(output_dir, exist_ok=True)
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    result = detector.detect(img)

    output_path = os.path.join(output_dir, f"{base_name}_detection.jpg")
    cv2.imwrite(output_path, cv2.cvtColor(result['visualization'], cv2.COLOR_RGB2BGR))
    mask_path = os.path.join(output_dir, f"{base_name}_mask.jpg")
    cv2.imwrite(mask_path, result['patch_mask'] * 255)

    fig_path = os.path.join(output_dir, f"{base_name}_analysis.jpg")
    plt.figure(figsize=(18, 6))
    plt.subplot(131)
    plt.title("Input Image")
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(132)
    plt.title("Detection Mask\n(White = suspicious regions)")
    plt.imshow(result['patch_mask'], cmap='gray')
    plt.axis('off')
    plt.subplot(133)
    plt.title("AVD-RGB Detection\n(Red = suspicious, Green = largest region)")
    plt.imshow(result['visualization'])
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(fig_path)
    plt.close()

    if csv_writer:
        csv_writer.writerow([
            base_name, config_name, result['is_adversarial'], result['confidence'],
            result['metrics']['spectral_confidence'], result['metrics']['uncertainty_score'],
            result['metrics']['patch_count'], result['metrics']['largest_patch_area'],
            result['metrics']['largest_patch_bbox']
        ])

    print(f"\n[✓] Processed {image_path}")


def batch_process_images(input_dir, output_dir="results", spectral=True, uncertainty=True):
    config_name = f"spectral={spectral}_uncertainty={uncertainty}"
    image_paths = glob(os.path.join(input_dir, "*"))
    image_paths = [p for p in image_paths if p.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not image_paths:
        print("Aucune image trouvée dans le dossier.")
        return

    detector = AVDRGB_Detector(use_spectral=spectral, use_uncertainty=uncertainty)
    csv_path = os.path.join(output_dir, f"results_{config_name}.csv")
    os.makedirs(output_dir, exist_ok=True)

    with open(csv_path, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Image", "Config", "Adversarial", "Confidence", "Spectral_Conf", "Uncertainty", "Patch_Count", "Largest_Area", "BBox"])
        for img_path in image_paths:
            try:
                analyze_and_save_results(img_path, detector, output_dir, csv_writer=writer, config_name=config_name)
            except Exception as e:
                print(f"[!] Erreur lors du traitement de {img_path} : {e}")

# Run all ablation configs
configs = [
    (True, True),   # Both
    (True, False),  # Spectral only
    (False, True)   # Uncertainty only
]
for spectral, uncertainty in configs:
    outdir = f"/content/drive/MyDrive/AVD-RGB/results_{spectral}_{uncertainty}"
    batch_process_images("/content/drive/MyDrive/AVD-RGB/images", outdir, spectral, uncertainty)
